# 1. IMPORTING LIBRARIES

In [14]:
import sys
import random
import gym
import numpy as np
from IPython.display import clear_output

from tqdm import tqdm
import time

! pip install gym[toy_text]

# 2. CREATING THE ENVIRONMENT
새로운 환경을 생성합니다.

In [15]:
env_name = "FrozenLake-v1"
env = gym.make(env_name)
env.seed(777)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


[777]

In [16]:
class Agent:
    def __init__(
        self,
        env: gym.Env,
    ):
        """ 초기화

        Args:
            env (gym.Env): openAI Gym 환경
            epsilon_decay (float): epsilon을 감소시키기 위한 스텝 크기
            lr (float): 학습률
            max_epsilon (float): epsilon의 최대값
            min_epsilon (float): epsilon의 최소값
            gamma (float): discount factor
        """

        # QTable 생성
        self.env = env

        self.state_size  = self.env.observation_space.n
        self.action_size = self.env.action_space.n

        self.lr = 0.9
        self.gamma = 0.99

        self.qtable = np.zeros((self.state_size, self.action_size))

    # EXPLORATION VS EXPLOITATION
    def get_action(self, state, epsilon):
        q_value = self.qtable[state,:]
        # 3. 현재 월드 상태에서 액션 a를 선택합니다. (s)
        # 만약 이 숫자가 엡실론보다 작으면 무작위 선택을 하게 됨 --> exploration
        if np.random.rand() <= epsilon:
            action = np.random.choice(self.action_size)

        ## 그렇지 않다면 --> exploitation (해당 상태에 대해 가장 큰 Q 값을 선택)
        else:
            action = np.argmax(q_value)

        return action

    def train_step(self, state, action, reward, next_state, done):

        curr_Q = self.qtable[state, action]
        next_Q = self.qtable[next_state, :]

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a) - Q(s,a)]
        self.qtable[state, action] = curr_Q + self.lr * (reward + gamma * np.max(next_Q) - curr_Q)

# 4. INITIALIZING THE Q-PARAMETERS

In [17]:
max_episodes = 5000  # 에이전트를 훈련시킬 전체 에피소드 수 설정.

max_steps = 99       # 에피소드 당 최대 스텝 수
gamma = 0.95         # 할인율
render = False       # 게임 환경을 표시할지 여부

# 탐험 매개변수
epsilon = 1.0        # 탐험률
max_epsilon = 1.0    # 시작 시 탐험 확률
min_epsilon = 0.01   # 최소 탐험 확률
decay_rate = 0.005   # 탐험 확률의 지수적 감소율

# 5. TRAINING LOOP

In [20]:
# train
agent = Agent(
    env,
#     memory_size,
#     batch_size,
#     epsilon_decay,
)

# 에이전트에게 제공된 모든 에피소드의 보상을 담을 리스트
scores = []

# 진행률 표시줄 설정
progress_bar = tqdm(range(max_episodes), desc="Episode Progress", unit="episode")

# 각 에피소드를 위한 반복문
for episode in progress_bar:
    state = agent.env.reset()
    episode_reward = 0
    done = False

    # 에피소드가 끝날 때까지 반복
    while not done:
        action = agent.get_action(state, epsilon)
        next_state, reward, done, _ = agent.env.step(action)
        agent.train_step(state, action, reward, next_state, done)

        state = next_state
        episode_reward += reward

        if done:
            scores.append(episode_reward)
            progress_bar.set_postfix(episode_reward=episode_reward)

    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

# 시간에 따른 평균 점수 출력
print ("Score over time: " +  str(sum(scores)/max_episodes))
print(agent.qtable) # Q-테이블 출력

# 천 개 에피소드당 평균 보상 계산 및 출력
count = 1000
rewards_per_thousand_episodes = np.split(np.array(scores),int(max_episodes/count))

print("\n\n********1000 episode마다 평균 리워드********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

Episode Progress: 100%|██████████| 5000/5000 [00:23<00:00, 215.76episode/s, episode_reward=1]


Score over time: 0.4112
[[3.69833783e-01 8.44168698e-02 5.65451225e-02 5.03575066e-02]
 [1.28788837e-03 1.14971828e-02 3.72820566e-03 4.38927229e-01]
 [2.92946452e-04 5.97829975e-03 1.26822667e-02 7.68191603e-02]
 [1.39443487e-05 2.16498650e-04 1.35338091e-03 2.25292421e-02]
 [2.02159411e-01 3.35962185e-02 1.55697516e-05 3.05334979e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.51793744e-09 1.43499861e-08 8.01912499e-02 9.42251876e-07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.08853048e-03 4.44968880e-02 1.02096697e-03 4.30687143e-01]
 [8.09889246e-04 6.02156572e-01 9.92926852e-03 1.40180983e-03]
 [3.90321346e-01 1.35641306e-03 9.89845767e-04 3.18962850e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.27095536e-02 6.80427351e-02 8.15706495e-01 6.62143979e-02]
 [3.37597079e-02 9.85459354e-01 9.62575651e-02 1.87196920e-02]
 [0.00000000e+00 0.00000000e+00

# 6.TEST
Q-테이블에 따라 최선의 행동을 선택하여 Frozen Lake 게임을 하는 에이전트 관찰해봅시다.

In [22]:
for episode in range(5):
    state = agent.env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("*******Episode ", episode+1, "*******\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps):
        # 현재 환경 상태를 화면에 표시
        clear_output(wait=True)
        # 현재 상태에 대해 가장 높은 Q-값을 가지는 행동 선택
        print(env.render(mode='ansi'))
        time.sleep(0.3)

        # 현재 상태에 대해 예상되는 최대 미래 보상을 가진 행동(인덱스)을 취함
        action = np.argmax(agent.qtable[state,:])

        new_state, reward, done, info = agent.env.step(action)

        if done:
            # 여기서는 에이전트가 목표에 도착했는지, 구멍에 빠졌는지 보기 위해 마지막 상태만 출력합니다.
            if new_state == 15:
                print("잘 도착했습니다 🏆")
            else:
                print("구덩이에 빠졌네요 ☠️")

            # 몇 단계가 걸렸는지 출력합니다.
            print("Number of steps", step)

            break
        state = new_state
agent.env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG

잘 도착했습니다 🏆
Number of steps 50
